#### Setup

In [ ]:
# general setup
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt

# bokeh
from bokeh.plotting import figure, show, output_file;
from bokeh.io import output_notebook;
output_notebook();

#### Read Data

In [ ]:
# data locations
scc_filename = '/home/tjc/research/flowmow/sentry281/nav-sci/proc/sentry281_20140805_1757_scc.csv'
sbe3_filename = '/home/tjc/research/flowmow/sentry281/nav-sci/raw/sbe3/sbe3.csv'
nortek_vsd_filename = '/home/tjc/research/flowmow/sentry281/nav-sci/raw/nortek/nortek_vsd.csv'
nortek_vvd_filename = '/home/tjc/research/flowmow/sentry281/nav-sci/raw/nortek/nortek_vvd.csv'
paros_filename = '/home/tjc/research/flowmow/sentry281/nav-sci/raw/parosKinsey/paros.csv'
ustrain_adv_filename = '/home/tjc/research/flowmow/sentry281/nav-sci/raw/ustrainAdv/ustrain_adv.csv'
ustrain_chassis_filename = '/home/tjc/research/flowmow/sentry281/nav-sci/raw/ustrain-chassis/ustrain_chassis.csv'

In [ ]:
# timestamp parser
parse = lambda x: pd.datetime.strptime(x, '%Y/%m/%d %H:%M:%S.%f')

In [ ]:
# read scc (sentry data)
scc = pd.read_csv(scc_filename, parse_dates=['timestamp'], date_parser=parse)
scc_epoch = []
for index, row in scc.iterrows():
    scc_epoch.append(int(row['timestamp'].to_pydatetime().replace(tzinfo=dt.timezone.utc).timestamp()))
scc['epoch'] = scc_epoch

In [ ]:
# read sbe3 (stinger temperature)
sbe3 = pd.read_csv(sbe3_filename, parse_dates=['timestamp'], date_parser=parse)
sbe3_epoch = []
for index, row in sbe3.iterrows():
    sbe3_epoch.append(row['timestamp'].to_pydatetime().replace(tzinfo=dt.timezone.utc).timestamp())
sbe3['epoch'] = sbe3_epoch

In [ ]:
# read nortek
nortek_vvd = pd.read_csv(nortek_vvd_filename, parse_dates=['timestamp'], date_parser=parse)
nortek_vvd_epoch = []
for index, row in nortek_vvd.iterrows():
    nortek_vvd_epoch.append(row['timestamp'].to_pydatetime().replace(tzinfo=dt.timezone.utc).timestamp())
nortek_vvd['epoch'] = nortek_vvd_epoch

#### Interpolate Positions

In [ ]:
# interpolate for position on sbe3
sbe3['x'] = np.interp(sbe3['epoch'].values, scc['epoch'].values, scc['x'].values)
sbe3['y'] = np.interp(sbe3['epoch'].values, scc['epoch'].values, scc['y'].values)

In [ ]:
# interpolate for position on nortek
nortek_vvd['x'] = np.interp(nortek_vvd['epoch'].values, scc['epoch'].values, scc['x'].values)
nortek_vvd['y'] = np.interp(nortek_vvd['epoch'].values, scc['epoch'].values, scc['y'].values)

In [ ]:
p = figure(width=500, height=300)
p.line(scc['x'], scc['y'])
show(p)

In [ ]:
p = figure(width=500, height=300)
p.line(sbe3['x'], sbe3['y'])
show(p)

In [ ]:
p = figure(width=500, height=300)
p.line(nortek_vvd['x'], nortek_vvd['y'])
show(p)

In [ ]:
ustrain_adv = pd.read_csv(ustrain_adv_filename, index_col = 'timestamp', parse_dates=['timestamp'], date_parser=parse)

In [ ]:
paros = pd.read_csv(paros_filename, index_col = 'timestamp', parse_dates=['timestamp'], date_parser=parse)

In [ ]:
paros.iloc[::100, 1].plot()

In [ ]:
ustrain_adv.iloc[::100, 1].plot();

In [ ]:
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()


p = figure(width=800, height=400, x_axis_type="datetime")
#p.line(ustrain_adv.iloc[::100].index.values, ustrain_adv.iloc[::100, 0])
show(p)

In [ ]:
p = figure(width=800, height=400, x_axis_type="datetime")
p.line(paros.iloc[::100].index.values, paros.iloc[::100, 0])
show(p)

In [ ]:
nortek_vvd = pd.read_csv(nortek_vvd_filename, index_col = 'timestamp', parse_dates=['timestamp'], date_parser=parse)
nortek_vvd.head()

In [ ]:
sbe3 = pd.read_csv(sbe3_filename, index_col = 'timestamp', parse_dates=['timestamp'], date_parser=parse)
sbe3.head()

In [ ]:
sbe3.describe()

In [ ]:
scc = pd.read_csv(scc_filename, parse_dates=['timestamp'], date_parser=parse)
scc.head()

In [ ]:
posix_timestamp = []
for index, row in scc.iterrows():
    posix_timestamp.append(int(row['timestamp'].to_pydatetime().replace(tzinfo=datetime.timezone.utc).timestamp()))

In [ ]:
scc['posix'] = posix_timestamp

In [ ]:
scc['posix'].tail()

In [ ]:
import datetime
test.replace(tzinfo=datetime.timezone.utc).timestamp()

In [ ]:
import numpy as np
#from bokeh.palettes import Spectral11
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()

In [ ]:
p = figure(width=500, height=300)
p.line(scc['x'], scc['y'])
show(p)

In [ ]:
%matplotlib notebook
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib as mpl

from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()

In [ ]:
store = pd.HDFStore('flowmow.h5')
store

In [ ]:
#scc = store.scc

In [ ]:
#scc.head()

In [ ]:
scc = store.scc.sort_values(by='T1')
norm = mpl.colors.Normalize(vmin=2.45, vmax=2.6)
cmap = mpl.cm.plasma
m = mpl.cm.ScalarMappable(norm=norm, cmap=cmap)
colors = ['#%02x%02x%02x' % (int(r),int(g),int(b)) for r,g,b,_ in 255*m.to_rgba(scc['T1'])]

# create a new figure
p = figure(tools='pan,wheel_zoom,box_zoom,reset', match_aspect=True, output_backend="webgl", lod_threshold=100)

# add a circle renderer with vecorized colors and sizes
p.circle(scc['x'], scc['y'], radius=1, fill_color=colors, fill_alpha=0.6, line_color=None)

# show the results
show(p)

In [ ]:
type((scc['T1']-scc['T1'].min())/scc['T1'].max())

In [ ]:
list(colors)[0]

In [ ]:
len(colors)

In [ ]:
mpl.cm.viridis(3)

In [ ]:
r

In [ ]:
norm = mpl.colors.Normalize(vmin=2.4, vmax=2.65)
cmap = mpl.cm.hot
m = mpl.cm.ScalarMappable(norm=norm, cmap=cmap)
colors = list(m.to_rgba(scc['T1']))

In [ ]:
list(test[1,0:3])